# Sentence Variational Adversarial Active Learning (S-VAAL)
@author: Tyler Bikaun

The following notebook intends to flesh out an initial proof concept of the coupling of S-VAE (Bowman <i>et al.</i> 2016; https://arxiv.org/abs/1511.06349) and VAAL (Sinha <i>et al.</i> 2019; https://arxiv.org/abs/1904.00370)

<b>Application:</b> Named Entity Task (NER)

<b>Architecture Diagram:</b>

### Standard Imports

In [1]:
import random
import numpy as np
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim

Tensor = torch.Tensor
torch.manual_seed(1)

## Testing Functions
These functions are used for spot testing code whilst developing. For example, building random sequences of tensors.
- [x] Build artificial sequence generator

In [2]:
class Tester:
    def __init__(self):
        self.pad_idx = 0
        self.special_chars_list = [self.pad_idx]
        self.label_space_size = 4

        
    def build_sequences(self, batch_size: int, max_seq_len: int) -> Tensor:
        """
        Builds tensor of specified size containing variable length, padded, sequences of integers
            
        Arguments
        ---------
            batch_size : int
                Number of sequences to generate
            max_seq_len : int
                Maximum length of sequences
        Returns
        -------
            sequences : tensor
                Tensor of generated sequences
            lengths : tensor
                Tensor of sequence lengths
        """
        seqs = list()
        for i in range(batch_size):
            # Generate random integer sequences
            # sequence must be at least 1 token long...
            seq = np.random.randint(low=1, high=100, size=(random.randint(1, max_seq_len),))
            # Add padding
            seq = np.concatenate((seq, np.ones(shape=(max_seq_len - len(seq)))*self.pad_idx), axis=None)
            seqs.append(seq)
        sequences = torch.LongTensor(seqs)
        lengths = torch.tensor([len(seq[seq != self.pad_idx]) for seq in sequences])
        
        print(f'Shapes - seq {sequences.shape} - lengths {lengths.shape}')
        
        return sequences, lengths
    
    def build_sequence_tags(self, sequences: Tensor, lengths: Tensor) -> Tensor:
        """
        Given a set of sequences, generates ground truth labels
        
        Labels need to be non-zero (otherwise get confused with special characters; currnetly only concerned about 0 = PAD)
        
        Arguments
        ---------
            sequences : tensor
                Tensor of generated sequences
            label_space_size : int
                Size of label space
        Returns
        -------
            X, lengths, y : list of tuples
                Artificial ground truth dataset
                    X dim : (seq len, batch size )
                    lengths dim : (batch size)
                    y dim : (batch size, 1)
        """
        
        dataset = list()    # stores batch of data (X, lens, y)
        
        global_label_list = list()
        
        for sequence in sequences:
            # Each 'token' in the sequence has a label mapping
            label_list = list()
            for token in sequence:
                if token != self.pad_idx:   # don't give a label to any padding...
                    label_list.append(random.randint(1, len(self.special_chars_list)+self.label_space_size))
                else:
                    label_list.append(self.pad_idx)
            
            global_label_list.append(torch.LongTensor(label_list))
        
        global_label_tensor = torch.stack(global_label_list)
        dataset.append((sequences, lengths, global_label_tensor))   # stack list of labels into tensors
        return dataset

In [3]:
# Test functionality
tester = Tester()
sequences, lengths = tester.build_sequences(batch_size=1, max_seq_len=10)
dataset = tester.build_sequence_tags(sequences=sequences, lengths=lengths)

Shapes - seq torch.Size([1, 10]) - lengths torch.Size([1])


In [4]:
for X, length, y in dataset:
    print(f'X {X.shape} ({X.dtype}) - {X}')
    print(f'y {y.shape} ({y.dtype}) - {y}')
    print(f'lengths {length.shape} ({length.dtype}) - {length}')

X torch.Size([1, 10]) (torch.int64) - tensor([[44,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
y torch.Size([1, 10]) (torch.int64) - tensor([[4, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
lengths torch.Size([1]) (torch.int64) - tensor([1])


### Utility Functions

- [ ] Build data preprocessor
- [ ] Build data loaders

In [5]:
# configuration for model building, training, evaluation process. This will be converted into yaml.
config = {'': ''}

### Models
<i>Model architectures</i><br>
<b>SVAE</b> - RNN<br>
<b>Discriminator</b> - FC NN<br>
<b>Task Learner</b> - RNN<br>
- [ ] SVAE
- [ ] Discriminator
- [ ] Task Learner

#### SVAE
To do:
 - [ ] 

In [6]:
class SVAE(nn.Module):
    """ Sentence Variational Autoencoder (Bowman et al. 2016)"""
    
    def __init__(self, vocab_size, embedding_size):
        super(SVAE, self).__init__()
        
        # TODO: fix dodgy vocab_size issue... this will be cleared up when utils implemented properly
        
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        
        self.max_sequence_length = 40    # arg
        self.pad_idx = 0
        self.eos_idx = vocab_size + 1
        self.sos_idx = vocab_size + 2
        self.unk_idx = vocab_size + 3
        
        self.vocab_size = vocab_size + 4
        
        self.z_dim = 8
        
        self.rnn_type = 'gru'
        self.bidirectional = False
        self.num_layers = 1
        self.hidden_size = 128
        
        self.embedding = nn.Embedding(self.vocab_size, embedding_size)
        self.word_dropout_rate = 0.1
        self.embedding_dropout = nn.Dropout(p=0.5)
        
        # set rnn type
        if self.rnn_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError()
        
        # init encoder-decoder RNNs (models are identical)
        self.encoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        # Initialisation of FC layers
        # These go from encoder to latent (z) space
        self.hidden2mean = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.hidden2logv = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.z2hidden = nn.Linear(self.z_dim, self.hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), self.vocab_size)
        
        # init partial loss function
        self.NLL = nn.NLLLoss(ignore_index=self.pad_idx, reduction='sum')   # TODO: REVIEW args
    
    
    def forward(self, input_sequence, length):
        """ Forward pass through VAE """
        
        batch_size = input_sequence.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)   # trick for packed padding
        input_sequence = input_sequence[sorted_idx]
        
        # ENCODER
        input_embedding = self.embedding(input_sequence)
#         print(input_embedding.shape)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        _, hidden = self._encode(packed_input)
        
        if self.bidirectional or 1 < self.num_layers:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size * self.hidden_factor)
        else:
            # .squeeze() -> Returns a tensor with all the dimensions of input of size 1 removed.
            print(f'hidden shape before squeeze {hidden.shape}')
#             hidden = hidden.squeeze()   # doesn't work? gives wrong dimension down stream...
            pass
            print(f'hidden shape after squeeze {hidden.shape}')

        
        # Reparameterisation trick!
        z, mean, logv, std = self.reparameterise(hidden, batch_size)
        
        # DECODER
        if 0 < self.word_dropout_rate:
            prob = torch.rand(input_sequence.size())

            if torch.cuda.is_available():
                prob = prob.cuda()

            prob[(input_sequence.data - self.sos_idx) * (input_sequence.data - self.pad_idx) == 0] = 1

            decoder_input_sequence = input_sequence.clone()
            
#             print(vocab_size)
#             print(self.unk_idx)
            decoder_input_sequence[prob < self.word_dropout_rate] = self.unk_idx

#             print(decoder_input_sequence)
            input_embedding = self.embedding(decoder_input_sequence)

        input_embedding = self.embedding_dropout(input_embedding)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        outputs, _ = self._decode(packed_input, hidden)
        
        # process outputs
        # Process outputs
        # Unpack padded sequence
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()

        # Project outputs to vocab
        # e.g. project hidden state into label space...
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
#         print(f'logp before view {logp.shape}\n')
#         print(f'b {b} s {s} no emb {self.embedding.num_embeddings}')
        logp = logp.view(b, s, self.embedding.num_embeddings)

        # logp - log posterior over label space; mean - tensor Gaussian mean, logv - tensor Gaussian variance, z - VAE latent space 
        return logp, mean, logv, z
    
    def to_var(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        return x
    
    def kl_anneal_function(self, anneal_function, step, k, x0):
        """
        
        
        """
        if anneal_function == 'logistic':
            return float(1/(1+np.exp(-k*(step-x0))))
        elif anneal_function == 'linear':
            return min(1, step/x0)
        
    def loss_fn(self, logp, target, mean, logv, anneal_function, step, k, x0):
        """
        SVAE loss function
        
        NLL - Negative Log Likelihood loss between predicted tags and ground truth
        KL_Loss - Evidence Lower Bound (ELBO)? - TODO: validate interpretation
        
        """
        
        # insert assertion
        
        # ALL THE CODE BELOW NEEDS REVIEW and piece wise implemented
        # Cut-off unnecessary padding from target and flatten
        target = target[:, :torch.max(length).item()].contiguous().view(-1)
        logp = logp.view(-1, logp.size(2))
        
        # Negative log likelihood
        NLL_loss = self.NLL(logp, target)
        
        # KL Divergence (or ELBO? TODO: determine which)
        KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
        KL_weight = self.kl_anneal_function(anneal_function, step, k, x0)
        
        return NLL_loss, KL_loss, KL_weight
    
    def reparameterise(self, hidden, batch_size):
        """ Implement reparameterisation trick (Kingma and Welling 2014) """
        
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv) 
        
        z = self.to_var(torch.randn([batch_size, self.z_dim]))
        return z * std + mean, mean, logv, std
    
    def _encode(self, x):
        """ x - pack padded sequence """
        return self.encoder_rnn(x)
    
    def _decode(self, x, hidden):
        """ x - pack padded sequence
            hidden - latent tensor"""
        return self.decoder_rnn(x, hidden)

In [7]:
# Testing functionality
vocab_size = 100
hidden_size = 128
svae = SVAE(vocab_size, hidden_size)

In [8]:
# Test forward pass of SVAE
svae_ff_seqs, svae_ff_lengths = Tester().build_sequences(batch_size=10, max_seq_len=40)

# Pass sequences and lengths into SVAE forward method
print(svae.forward(svae_ff_seqs, svae_ff_lengths))

Shapes - seq torch.Size([10, 40]) - lengths torch.Size([10])
hidden shape before squeeze torch.Size([1, 10, 128])
hidden shape after squeeze torch.Size([1, 10, 128])
(tensor([[[-4.7999, -4.7079, -4.6882,  ..., -4.5423, -4.7034, -4.8623],
         [-4.6320, -4.7786, -4.9867,  ..., -4.4183, -4.8525, -4.4922],
         [-4.6727, -4.9489, -4.7301,  ..., -4.4110, -4.8230, -4.4724],
         ...,
         [-4.6752, -4.6746, -4.5652,  ..., -4.5551, -4.6831, -4.5759],
         [-4.6752, -4.6746, -4.5652,  ..., -4.5551, -4.6831, -4.5759],
         [-4.6752, -4.6746, -4.5652,  ..., -4.5551, -4.6831, -4.5759]],

        [[-4.7635, -5.1990, -4.4356,  ..., -4.6312, -4.9385, -4.2988],
         [-4.5988, -4.7531, -4.5594,  ..., -4.7206, -4.6209, -4.8014],
         [-4.7039, -4.6724, -4.7143,  ..., -4.8227, -4.6895, -4.7346],
         ...,
         [-4.6752, -4.6746, -4.5652,  ..., -4.5551, -4.6831, -4.5759],
         [-4.6752, -4.6746, -4.5652,  ..., -4.5551, -4.6831, -4.5759],
         [-4.6752, -4.

In [9]:
# Reviewing components
print(f'Encoder: {svae.encoder_rnn}')
print(f'Decoder: {svae.decoder_rnn}')
print(f'hidden2mean: {svae.hidden2mean}')
print(f'hidden2logv: {svae.hidden2logv}')
print(f'z2hidden: {svae.z2hidden}')
print(f'outputs2vocab: {svae.outputs2vocab}')

Encoder: GRU(128, 128, batch_first=True)
Decoder: GRU(128, 128, batch_first=True)
hidden2mean: Linear(in_features=128, out_features=8, bias=True)
hidden2logv: Linear(in_features=128, out_features=8, bias=True)
z2hidden: Linear(in_features=8, out_features=128, bias=True)
outputs2vocab: Linear(in_features=128, out_features=104, bias=True)


#### Task Learner
Using PyTorch tutorial implementation (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) for rapid development (will implement SoTA in the future)<br><br>
To do:
 - [ ] Make batch based to suit other models rather than training on singletons

In [46]:
class TaskLearner(nn.Module):
    """ Task learner for NER """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(TaskLearner, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)    # expects input (batch, seq, feature)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, input_sequences, lengths):
        """
        Pack padded sequence and sorted lengths used to optimise forward pass through RNN
        
        
        input_sequence : tensor
            (batch, seq, feature)
        
        """
#         print(input_sequences)
    
        batch_size = input_sequences.size(0)
        sorted_lengths, sorted_idx = torch.sort(lengths, descending=True)
        input_sequences = input_sequences[sorted_idx]
#         print(input_sequences)
        
        input_embeddings = self.word_embeddings(input_sequences)
#         print(input_embeddings)
        
        packed_input = rnn_utils.pack_padded_sequence(input_embeddings, sorted_lengths.data.tolist(), batch_first=True)
        
        lstm_out, _ = self.lstm(packed_input)
        
        # Unpack padded sequence
        padded_outputs = rnn_utils.pad_packed_sequence(lstm_out, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()
        
        print(b, '-', s)
        
        # project into label space
        tag_space = self.hidden2tag(padded_outputs.view(-1, padded_outputs.size(2)))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [47]:
# Testing functionality
tasklearner = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=104, tagset_size=4)
# Generating data for testing
seqs, lens = tester.build_sequences(batch_size=2, max_seq_len=10)   # Tester initialised at start of notebook
dataset = tester.build_sequence_tags(sequences=seqs, lengths=lens)

Shapes - seq torch.Size([2, 10]) - lengths torch.Size([2])


In [48]:
# Testing forward pass functionality
for X, lens, y in dataset:
#     print(X, lens, y)
    forward_out = tasklearner.forward(input_sequences=X, lengths=lens)
    print(f'Forward pass output shape: {forward_out.shape}')

2 - 9
Forward pass output shape: torch.Size([18, 4])


In [49]:
# Generate vocab off of generated sequences
vocab = list()
for seq in seqs:
      vocab.extend(seq.tolist())
vocab = list(set(vocab))
print(vocab)

[0, 98, 70, 71, 10, 13, 14, 19, 52, 55, 88, 57, 90, 27, 95]


In [50]:
# Test training routine
# vocab_size is max int in vocab ints + 1 as 0 is included...
model = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=max(vocab)+1, tagset_size=6)   # tagset size = special characters + number of class types (0 to n_class - 1)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [51]:
with torch.no_grad():
    print(seqs.shape)
    # forward pass to get output scores
    tag_scores = model(seqs, lens)
    print(tag_scores.shape)
#     print(tag_scores)

torch.Size([2, 10])
2 - 9
torch.Size([18, 6])


In [83]:
# need to fix to be batch based rather than individual sequence
for epoch in range(5):
    
    for batch_seqs, batch_lens, batch_tags in dataset:
        print(f'\nEpoch: {epoch}')
#         print(f'Shapes | Seq: {batch_seqs.shape} Lengths: {batch_lens.shape} Tags: {batch_tags.shape}')
        
#         print(batch_seqs)
        
        # get max length of longest sequence in batch (use to filter padding on tags)
        sorted_lengths, _ = torch.sort(batch_seqs, descending=True)   # longest seq at index 0
        longest_seq = sorted_lengths[0].data.numpy()
        longest_seq_nopad = longest_seq[longest_seq != 0]
        longest_seq_len = len(longest_seq_nopad)
        
        model.zero_grad()
        tag_scores = model(batch_seqs, batch_lens)
        # Remove padding from tags as tag_scores go through pack padded sequence (variable length sequences)
        print(batch_tags)
        
        batch_tags = torch.stack([tags[:longest_seq_len] for tags in batch_tags])
        
        batch_tags = batch_tags.view(-1)
        
#         batch_tags = batch_tags[:longest_seq_len-1]
        print(batch_tags)
        
#         batch_tags = batch_tags.view(-1)[batch_tags.view(-1).nonzero()].view(-1)
        print(batch_tags)

        loss = loss_function(tag_scores, batch_tags)
        loss.backward()
        optimizer.step()
        
        print(f'Loss:',loss.data.detach())


Epoch: 0
2 - 9
tensor([[5, 5, 5, 3, 5, 4, 3, 4, 5, 0],
        [3, 3, 4, 3, 4, 0, 0, 0, 0, 0]])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
Loss: tensor(1.8186)

Epoch: 1
2 - 9
tensor([[5, 5, 5, 3, 5, 4, 3, 4, 5, 0],
        [3, 3, 4, 3, 4, 0, 0, 0, 0, 0]])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
Loss: tensor(1.7850)

Epoch: 2
2 - 9
tensor([[5, 5, 5, 3, 5, 4, 3, 4, 5, 0],
        [3, 3, 4, 3, 4, 0, 0, 0, 0, 0]])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
Loss: tensor(1.7526)

Epoch: 3
2 - 9
tensor([[5, 5, 5, 3, 5, 4, 3, 4, 5, 0],
        [3, 3, 4, 3, 4, 0, 0, 0, 0, 0]])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
tensor([5, 5, 5, 3, 5, 4, 3, 4, 5, 3, 3, 4, 3, 4, 0, 0, 0, 0])
Loss: tensor(1.7213)

Epoch: 4
2 - 9
tensor([

In [24]:
# need to fix to be batch based rather than singleton
with torch.no_grad():
    inputs, _, _ = dataset[0]
    tag_scores = model(inputs)

    print(tag_scores)

TypeError: forward() missing 1 required positional argument: 'lengths'

#### Discriminator
To do:
 - [ ] 

In [ ]:
class Discriminator(nn.Module):
    """ Adversary architecture for discriminator module 
    
    Tensor needs to be FloatTensor; sequences and tag sets are LongTensor
    """
    
    def __init__(self, z_dim=8):
        super(Discriminator, self).__init__()
        
        self.z_dim = z_dim    # latent space dimension (will be the same shape as the encoder output from VAE)
        
        self.net = nn.Sequential(
                                nn.Linear(z_dim, 128),
                                nn.ReLU(True),
                                nn.Linear(128, 128),
                                nn.ReLU(True),
                                nn.Linear(128,1),
                                nn.Sigmoid()
                                )
        # Exe
        self.weight_init()
        
    def weight_init(self):
        """ Weight initialisation
        
        Using Xavier uniform initialisation rather than Kaiming (I think that is more focused for CV? TODO: investigate)
        See: https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html
        """
        for block in self._modules:
            for m in self._modules[block]:
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.01)
    
    def forward(self, z):
        """ Forward pass through discriminator
        
        Arguments
        --------
            z : tensor
                Tensor derived from SVAE latent space
        """
        return self.net(z.type(torch.FloatTensor))

In [ ]:
# Testing functionality
z_dim = 8
discriminator = Discriminator(z_dim=z_dim)

# Pass random integer tensor through forward pass of discriminator
rand_tensor = torch.randint(low=0, high=100,size=(z_dim,))
print(f'Input: {rand_tensor}')
print(f'Output: {discriminator.forward(rand_tensor)}')

In [ ]:
# Testing training functionality for discriminator
dsc_bce_loss = nn.BCELoss()
dsc_optim = optim.Adam(discriminator.parameters(), lr=0.001)

discriminator.train()

preds = discriminator(rand_tensor)
real_labels = torch.ones(preds.size(0))
loss = dsc_bce_loss(preds, real_labels)

print(loss)

## Sampler Routine
Active learning based sample selection for task learner

In [ ]:
# Code copied from VAAL and modified for sequence data
class Sampler:
    """ Adversary sampler """
    def __init__(self, budget):
        self.budget = budget
        
    def sample(self, vae, discriminator, data, cuda):
        """ Selective sampling algorithm
        
        Arguments
        ---------
            vae : torch model
                VAE model
            discriminator : torch model
                discriminator model
            data : tensor
                Image data
            cuda : boolean
                GPU flag
        Returns
        -------
            querry_pool_indices: int, list
                List of indices corresponding to sorted (top-K) samples to be sampled from
        """
        all_preds = []
        all_indices = []

        for images, _, indices in data:
            if cuda:
                images = images.cuda()

            with torch.no_grad():
                _, _, mu, _ = vae(images)
                preds = discriminator(mu)

            preds = preds.cpu().data
            all_preds.extend(preds)
            all_indices.extend(indices)

        all_preds = torch.stack(all_preds)
        all_preds = all_preds.view(-1)
        # need to multiply by -1 to be able to use torch.topk 
        all_preds *= -1

        # select the points which the discriminator things are the most likely to be unlabeled
        _, querry_indices = torch.topk(all_preds, int(self.budget))
        querry_pool_indices = np.asarray(all_indices)[querry_indices]

        return querry_pool_indices

In [ ]:
# Testing functionality
sampler = Sampler(budget=10)

### Training Routine
<i> Pseudo code</i>

```python
for epoch in max_epochs:
        train(task learner)
            get preds
            calc loss
            zero grads
            backpropigate loss
            update model parameters
        for step in max_steps:
            train(SVAE)
        for step in max_steps:
            train(discriminator)
```
To do:
 - [ ] Implement training cycle for task learn
 - [ ] Implement training cycle for VAE
 - [ ] Implement training cycle for discriminator

In [ ]:
class Solver:
    def __init__(self, task_learner, vae, discriminator):
        self.epochs = 2
        self.svae_steps = 2
        self.discriminator_steps = 2
        
        # Get models
        # init else where?
        self.task_learner =  TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=max(vocab)+1, tagset_size=4)
        self.svae = SVAE(vocab_size=max(vocab)+1, embedding_size=128)
        self.discriminator = Discriminator(z_dim=10)   # z_dim will be the latent output of the vae encoder... currently just the seq length
        
        # Loss functions
        self.nll_loss = nn.NLLLoss()   # used in: TL
        self.bce_loss = nn.BCELoss()   # used in: Discriminator
        self.xxx_loss = 0    # used in: SVAE

        # optimiser
        self.optim_lr = 0.1
        self.tl_optim = optim.SGD(model.parameters(), lr=self.optim_lr)   # todo: revisit learning rate
        self.svae_optim = None
        self.disc_optim = optim.Adam(self.discriminator.parameters(), lr=0.01)   # todo: revisit learning rate
        
        # Will need to implement this in the future, need to build generator function rather than rely on Torch implementationss?
        self.data_loader = 'generator'
        self.seqs, self.lens = tester.build_sequences(batch_size=10, max_seq_len=10)   # Tester initialised at start of notebook
        self.dataset = tester.build_sequence_tags(sequences=self.seqs, lengths=self.lens, label_space_size=4)
        
        # Will need to implement this in the future, inherit the Sampler class?
        self.sampler = 'sampler'
        
    def train(self):
        """ Performs model training """
        
        # turn on .train() mode
        self.svae.train()
        self.task_learner.train()
        self.discriminator.train()
        
        for epoch in range(self.epochs):
            print(f'\nEpoch: {epoch}')
            
            # these will be batches in the future, not single sequence/tag sets...
            # build generator to wrap around sequence/tag sets and then just call next() on them at each epoch to get a batch...
            # need to build that in the future to partition based on the unlabelled/labelled splits for AL experiments
            for seqs, lens, tags in self.dataset:
                print(f'Dataset shapes\nSeqs:{seqs.shape}\tLens:{lens.shape}\tTags:{tags.shape}')
            
                # Train Task Learner
                self.tl_train_step(seqs, tags)
            
            
                # Train VAE
                for step in range(self.svae_steps):
                    print(f'SVAE Step: {step}')
                    self.svae_train_step(seqs, lens, tags)

                # Train Discriminator
                for step in range(self.discriminator_steps):
                    print(f'Discriminator Step: {step}')

                    self.disc_train_step(seqs)
            clear_output(wait=True)
                
    def tl_train_step(self, seqs, tags):
        """
        Perform training step of task learner 
        
        Tensor dtype is LongTensor
        """
#         print(seq.dtype)
#         assert seq.dtype == 'torch.int64'
#         assert tags.dtype == 'torch.int64'
        
        # zero grad as they accumulate in PyTorch
        self.task_learner.zero_grad()
        preds = self.task_learner(seqs)
        tl_loss = self.nll_loss(preds, tags.view(-1).type(torch.LongTensor))   # flatten tags as they are a 1D tensor rather than 2D
        tl_loss.backward()
        self.tl_optim.step()
        
        print(f'Task learning loss: {loss.data:0.4f}')
    
    def svae_train_step(self, seqs, lengths, tags):
        """
        Perform training step of sentence variational autoencoder
        
        Tensor type is LongTensor
        """
#         print(self.svae)
        
        # Forward pass
        logp, mean, logv, z = self.svae(seq, lengths)
        
        # Loss calculation
        NLL_loss, KL_loss, KL_weight = loss_fn(logp, tags, lengths, mean, logv, anneal_function, step, k, x0)
        
        
        
        
    
    def disc_train_step(self, seqs):
        """ Perform training step of adversarial discriminatory 
        
        Note: The loss function will be an aggregate across labelled and unlabelled samples (currently just labelled)
        
        Tensor dtype is FloatTensor for discriminator
        """
        
#         assert seq.dtype == 'torch.float64'
        
#         print(seq)
#         print(seq.dtype)
        
        preds = self.discriminator(seqs)
#         print(preds)
        real_labels = torch.ones(preds.size(0))
        disc_loss = self.bce_loss(preds, real_labels)
        disc_loss.backward()
        self.disc_optim.step()
        
        print(f'Discriminator loss: {loss.data}')
    

In [ ]:
# Test training 
slvr = Solver(task_learner='tl', vae='vae', discriminator='disc')
slvr.train()

### Inference Routine
To do:
 - [ ] 


### Sampling Routine
To do:
 - [ ] 